In [4]:
import sys
import datetime as dt
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [5]:
data_path = sys.path[0] + "/../data/Customer_Transaction.csv"
rfm_df = pd.read_csv(data_path)
rfm_df.sample(5)

,CustomerID,Recency,Frequency,Monetary
81715,841506,92,0.25,18750
84261,621292,31,0.25,23750
26196,714717,92,0.25,18750
90649,506922,92,0.20,15000
45519,902265,31,0.25,18750


In [6]:
quantile_dict = rfm_df.quantile([0.25, 0.5, 0.75], numeric_only = True).to_dict()
quantile_dict

{'CustomerID': {0.25: 417656.0, 0.5: 660600.0, 0.75: 868292.0},
 'Recency': {0.25: 31.0, 0.5: 62.0, 0.75: 92.0},
 'Frequency': {0.25: 0.2, 0.5: 0.25, 0.75: 0.25},
 'Monetary': {0.25: 15833.0, 0.5: 18750.0, 0.75: 23750.0}}

In [7]:
def recency_score(dict_var, key, value):
    if (value < dict_var[key][0.25]):
        return 4
    if (value < dict_var[key][0.5]):
        return 3
    if (value < dict_var[key][0.75]):
        return 2
    else: return 1

In [8]:
def frequency_and_monetary_score(dict_var, key, value):
    if (value <= dict_var[key][0.25]):
        return 1
    if (value <= dict_var[key][0.5]):
        return 2
    if (value <= dict_var[key][0.75]):
        return 3
    else: return 4

In [9]:
rfm_df["RecencyScore"] = rfm_df["Recency"].apply(lambda x: recency_score(quantile_dict, "Recency", x))
rfm_df["FrequencyScore"] = rfm_df["Frequency"].apply(lambda x: frequency_and_monetary_score(quantile_dict, "Frequency", x))
rfm_df["MonetaryScore"] = rfm_df["Monetary"].apply(lambda x: frequency_and_monetary_score(quantile_dict, "Monetary", x))
rfm_df.sample(5)

,CustomerID,Recency,Frequency,Monetary,RecencyScore,FrequencyScore,MonetaryScore
45147,747068,31,0.50,33750,3,4,4
50333,689029,31,0.25,21250,3,2,3
31676,977095,92,0.25,18750,1,2,2
110297,745678,62,0.25,31250,2,2,4
95200,467858,31,0.20,21000,3,1,3


In [10]:
rfm_df["RFMScore"] = rfm_df["RecencyScore"].astype(str) + rfm_df["FrequencyScore"].astype(str) +rfm_df["MonetaryScore"].astype(str)
rfm_df.sample(5)

,CustomerID,Recency,Frequency,Monetary,RecencyScore,FrequencyScore,MonetaryScore,RFMScore
92527,984303,31,0.25,23750,3,2,3,323
105227,278060,31,0.33,31666,3,4,4,344
2231,972676,31,0.25,18750,3,2,2,322
108507,539898,62,0.20,15000,2,1,1,211
31183,1046740,31,0.33,35000,3,4,4,344


In [11]:
customer_lst = ["Lost", "Hibernating", "About To Sleep", "Needs Attention", "At Risk", "Can\'t Lose Them", "New Customers", "Promising", "Potential Loyalist", "Loyal Customers", "Champions" ]
segment_map = {
    "1[1-2][1-2]": customer_lst[0] ,
    "2[1-2][1-2]": customer_lst[1],
    "2[3-4][1-2]": customer_lst[2],
    "1[3-4][1-2]": customer_lst[3],
    "2[1-4][3-4]": customer_lst[4],
    "1[1-4][3-4]": customer_lst[5],
    "[3-4][1-2][1-2]": customer_lst[6],
    "[3-4][1-2][3-4]": customer_lst[7],
    "[3-4][3-4][1-2]": customer_lst[8],
    "[3-4][3-4][3]": customer_lst[9],
    "[3-4][3-4][4]": customer_lst[10] 
}

In [12]:
rfm_df["CustomerSegmentation"] = rfm_df["RFMScore"].replace(segment_map, regex = True)
rfm_df.sample(5)

,CustomerID,Recency,Frequency,Monetary,RecencyScore,FrequencyScore,MonetaryScore,RFMScore,CustomerSegmentation
113706,671848,92,0.25,18750,1,2,2,122,Lost
10343,757138,31,0.25,21250,3,2,3,323,Promising
14357,1009758,31,0.25,26250,3,2,4,324,Promising
38308,1043457,62,0.33,25000,2,4,4,244,At Risk
95456,328599,92,0.17,12500,1,1,1,111,Lost


In [13]:
customer_segment_df = rfm_df.groupby("CustomerSegmentation", as_index = False).agg({"CustomerID": "count", "Monetary": "sum"})
customer_segment_df = customer_segment_df.rename(columns = {"CustomerID": "TotalUsers", "Monetary": "Total$"})
customer_segment_df["PercentageSize"] = round(customer_segment_df["TotalUsers"] / customer_segment_df["TotalUsers"].sum() * 100, 2)
customer_segment_df["Percentage$"] = round(customer_segment_df["Total$"] / customer_segment_df["Total$"].sum() * 100, 2)
# customer_segment_df.sort_values(by = "CustomerSegmentation", key = lambda x: x.map(customer_lst))
customer_segment_df = customer_segment_df.sort_values(by = "TotalUsers", ascending = False)
customer_segment_df.reset_index().drop("index", axis = 1)

,CustomerSegmentation,TotalUsers,Total$,PercentageSize,Percentage$
0,Lost,23716,379698568,20.85,15.42
1,New Customers,17871,283291277,15.71,11.51
2,Hibernating,17675,281967192,15.54,11.45
3,Can't Lose Them,17117,416674954,15.04,16.92
4,At Risk,13138,330861784,11.55,13.44
5,Promising,11871,286507802,10.43,11.64
6,Champions,11783,471663092,10.36,19.16
7,Loyal Customers,373,8140365,0.33,0.33
8,Potential Loyalist,187,2548202,0.16,0.10
9,About To Sleep,34,517973,0.03,0.02


save_data_path = sys.path[0] + "/../data/"
customer_segment_df.to_excel(save_path + "data_customer_segment_rfm.xlsx")